In [ ]:
# So Colab can use the python files...
!git clone --single-branch -b develop https://github.com/muellevin/Studienarbeit.git
%cd Studienarbeit/Detection_training

# 1. Setup Paths

In [ ]:
import os
import Tensorflow.scripts.Paths as pp

CUSTOM_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'

paths = pp.paths
paths.setup_paths()

CHECKPOINT_PATH = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'checkpoint')
OUTPUT_PATH = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'export')
SAVED_MODEL = os.path.join(OUTPUT_PATH, 'saved_model')
TFLITE_EXPORT = os.path.join(OUTPUT_PATH, 'tfliteexport')
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH, exist_ok=True)

PIPELINE_CONFIG = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'pipeline.config')

# 2. Download pretrained Tensorflow Model and install TFOD

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
# Download TensorFlow Object detection library if not available
if not os.path.exists(os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection')):
    command = "git clone https://github.com/tensorflow/models {}".format(paths.APIMODEL_PATH)

    !{command}

In [ ]:
# For Coral Edge TPU
! git clone https://github.com/google-coral/tutorials.git

! cp -r tutorials/docker/object_detection/scripts/* models/research/

Install TFOD (TensorFlow Object Detection)

In [ ]:
protoc_ver = '3.20.1'
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url=f'https://github.com/protocolbuffers/protobuf/releases/download/v{protoc_ver}/protoc-{protoc_ver}-win64.zip'
    wget.download(url)
    !move protoc-{protoc_ver}-win64.zip {os.path.join(paths.PROTOC_PATH, f'protoc.zip')}
    !cd {paths.PROTOC_PATH} && tar -xf protoc.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths.PROTOC_PATH, 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Verify that everything is setup correctly.  
The script should execute with 'OK (skipped=1)'

In [ ]:
%pip install tensorflow
%pip install tensorflow_addons
%pip install gin-config
%pip install pyyaml

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
import object_detection

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths.PRETRAINED_MODEL_PATH}
    !cd {paths.PRETRAINED_MODEL_PATH} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths.PRETRAINED_MODEL_PATH}
    !cd {paths.PRETRAINED_MODEL_PATH} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

# 3. Create Label Map

In [ ]:
with open(pp.LABELMAP, 'w') as f:
    for label in pp.LABELS:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 4. Upload dataset to Colab

Upload the dataset to google drive

In [ ]:
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/Studienarbeit'

if os.name =='posix':
    from google.colab import drive

    drive.mount('/content/drive')

    import shutil
    shutil.copy(os.path.join(GOOGLE_DRIVE_PATH, pp.DATASET_NAME), paths.IMAGE_PATH)

Now your google drive is connected and you have to copy the dataset into the image directory

Verify that dataset is available

In [ ]:
if not os.path.isfile(pp.DATASET):
  print('No dataset available')
else:
  print('Dataset available')

# 5. Create TF records

In [ ]:
if os.path.exists(pp.DATASET):
  !tar -zxvf {pp.DATASET}

In [ ]:
!python {pp.TF_RECORD_SCRIPT} -img_path {paths.TRAINSET_PATH} -output_path {pp.TRAINSET_RECORD_PATH}
!python {pp.TF_RECORD_SCRIPT} -img_path {paths.TESTSET_PATH} -output_path {pp.TESTSET_RECORD_PATH}

# 6. Copy Model Config to Training Folder

In [ ]:
if os.name =='posix':
    !cp {os.path.join(paths.PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'pipeline.config')} {CHECKPOINT_PATH}
if os.name == 'nt':
    !copy {os.path.join(paths.PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'pipeline.config')} {CHECKPOINT_PATH}

# 7. Update Config for Transfer Learning

In [ ]:
import tensorflow as tf
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(PIPELINE_CONFIG, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

pipeline_config.model.ssd.num_classes = len(pp.LABELS)
pipeline_config.train_config.batch_size = 16
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths.PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= pp.LABELMAP
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [pp.TRAINSET_RECORD_PATH]
pipeline_config.eval_input_reader[0].label_map_path = pp.LABELMAP
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [pp.TESTSET_RECORD_PATH]
pipeline_config.model.ssd.box_predictor.convolutional_box_predictor.kernel_size = 3
pipeline_config.model.ssd.box_predictor.convolutional_box_predictor.use_depthwise = True
pipeline_config.model.ssd.feature_extractor.use_depthwise = True
# Quantization Aware Training
pipeline_config.graph_rewriter.quantization.delay = 0
pipeline_config.graph_rewriter.quantization.weight_bits = 8
pipeline_config.graph_rewriter.quantization.activation_bits = 8

config_text = text_format.MessageToString(pipeline_config)

with tf.io.gfile.GFile(PIPELINE_CONFIG, "wb") as f:
    f.write(config_text)

# 8. Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, CHECKPOINT_PATH, PIPELINE_CONFIG)
print(command)

In [ ]:
!{command}

# 9. Freezing the Graph

In [ ]:
FREEZE_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'exporter_main_v2.py ')
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT , PIPELINE_CONFIG, CHECKPOINT_PATH, OUTPUT_PATH)

In [ ]:
!{command}

# 10 Conversion to tflit and edge-tpu

In [ ]:
TFLITE_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'export_tflite_graph_tf2.py ')

command = "python {} --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(TFLITE_SCRIPT , PIPELINE_CONFIG, CHECKPOINT_PATH, TFLITE_EXPORT)

!{command}

In [ ]:
import tensorflow as tf
import numpy as np
FROZEN_TFLITE_PATH = os.path.join(TFLITE_EXPORT, 'saved_model')
TFLITE_MODEL = os.path.join(TFLITE_EXPORT, 'saved_model', 'detect.tflite')

# converter = tf.lite.TFLiteConverter.from_saved_model(FROZEN_TFLITE_PATH)


# def representative_dataset():
#  for _ in range(100):
#   data = np.random.rand(1, 320, 320, 3)
#   yield [data.astype(np.float32)]

# # This enables quantization
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# # This sets the representative dataset for quantization
# converter.representative_dataset = representative_dataset
# # This ensures that if any ops can't be quantized, the converter throws an error
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# # For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
# converter.target_spec.supported_types = [tf.int8]
# # These set the input and output tensors to uint8 (added in r2.3)
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8

# tflite_model = converter.convert()

# """
# ## Save the quantized model
# """

# with open(TFLITE_MODEL, 'wb') as f:
#  f.write(tflite_model)


command = "tflite_convert \
--saved_model_dir={} \
--output_file={} \
--enable_v1_converter\
--input_shapes=1,320,320,3 \
--input_arrays=normalized_input_image_tensor \
--output_arrays='TFLite_Detection_PostProcess',TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
--inference_type QUANTIZED_INT8 \
--mean_values=128 \
--std_dev_values=128 \
--allow_nudging_weights_to_use_fast_gemm_kernel=true \
--allow_custom_ops".format(FROZEN_TFLITE_PATH, TFLITE_MODEL)

!{command}

interpreter = tf.lite.Interpreter(model_path = TFLITE_MODEL)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input Shape:", input_details[0]['shape'])
print("Input Type:", input_details[0]['dtype'])
print("Output Shape:", output_details[0]['shape'])
print("Output Type:", output_details[0]['dtype'])

In [ ]:
# test it
test_img = os.path.join(paths.IMAGE_PATH, 'test_image.jpg')
command = f'python {pp.EDGE_TPU_DETECT} -m {TFLITE_MODEL} -l {pp.LABEL_FILE_LITE} -i {pp.TEST_IMAGE} -o {test_img} -t 0.2'
print(command)
!{command}

In [ ]:
from object_detection.utils import label_map_util

category_index = label_map_util.create_category_index_from_labelmap(
    pp.LABELMAP)
f = open('labels.txt', 'w')
for class_id in range(1, 91):
  # if class_id not in category_index:
  #   f.write('???\n')
  #   continue
  # name = category_index[class_id]['name']
  name = str(class_id)
  f.write(name+'\n')
f.close()

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

_TFLITE_MODEL_WITH_METADATA_PATH = os.path.join(FROZEN_TFLITE_PATH, "model_with_metadata.tflite")

writer = object_detector.MetadataWriter.create_for_inference(
    writer_utils.load_file(TFLITE_MODEL), input_norm_mean=[127.5], 
    input_norm_std=[127.5], label_file_paths=['labels.txt'])
writer_utils.save_file(writer.populate(), _TFLITE_MODEL_WITH_METADATA_PATH)

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Load the frozen graph
model_path = FROZEN_TFLITE_PATH
model = tf.saved_model.load(model_path)
inputs = model.signatures['serving_default'].inputs
outputs = model.signatures['serving_default'].outputs

# Set parameters for preprocessing, postprocessing, and visualization
input_width = 320
input_height = 320
input_format = 'rgb'
score_threshold = 0.9
class_names = [str(i+1) for i in range(90)]  # replace with your own class names

# Define functions for preprocessing, postprocessing, and visualization
def preprocess_image(image):
    image = cv2.resize(image, (input_width, input_height))
    image = np.expand_dims(image, axis=0)
    return image

def postprocess_output(output):
    boxes = output['detection_boxes'][0].numpy()
    scores = output['detection_scores'][0].numpy()
    class_ids = output['detection_classes'][0].numpy().astype(np.int32)
    mask = scores > score_threshold
    boxes = boxes[mask]
    scores = scores[mask]
    class_ids = class_ids[mask]
    boxes[:, 0] *= input_height
    boxes[:, 1] *= input_width
    boxes[:, 2] *= input_height
    boxes[:, 3] *= input_width
    detections = []
    for box, score, class_id in zip(boxes, scores, class_ids):
        detection = {
            'box': box.tolist(),
            'score': score.tolist(),
            'class_id': class_id.tolist()
        }
        detections.append(detection)
    return detections

def visualize_output(image, detections):
    for detection in detections:
        box = detection['box']
        score = detection['score']
        class_id = detection['class_id']
        color = (0, 255, 0)  # green
        label = f"{class_names[class_id]}: {score:.2f}"
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        cv2.rectangle(image, (y1, x1), (y2, x2), color, thickness=2)
        cv2.putText(image, label, (y1, x1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, thickness=2)
    return image

# Define the input and output directories
input_dir = paths.IMAGE_PATH
output_dir = 'test'
os.makedirs(output_dir, exist_ok=True)

# Loop over the input images and perform object detection
for filename in os.listdir(input_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Load the input image
        image_path = os.path.join(input_dir, filename)
        image = cv2.imread(image_path)

        # Preprocess the input image
        input_image = preprocess_image(image)

        # Run object detection on the input image
        output = model(input_image)

        # Postprocess the output detections
        detections = postprocess_output(output)

        # Visualize the output detections
        output_image = visualize_output(cv2.resize(image, (input_width, input_height)), detections)

        # Save the output image
        output_path = os.path.join(output_dir, filename)
        cv2.imwrite(output_path, output_image)


In [ ]:
# test it
test_img = os.path.join(paths.IMAGE_PATH, 'test_image.jpg')
command = f'python {pp.EDGE_TPU_DETECT} -m yolov8n_full_integer_quant_edgetpu.tflite -l {pp.LABEL_FILE_LITE} -i {pp.TEST_IMAGE} -o {test_img} -t 0.1'
print(command)
!{command}

In [ ]:
command = f'python {pp.TESTING_DETECT_SCRIPT} \
            --modeldir {FROZEN_TFLITE_PATH} \
            --graph model_with_metadata.tflite \
            --labels labels.txt \
            --image {pp.TEST_IMAGE} \
            --save_results \
            --threshold 0.001'
!{command}

# test not pycoral


For Coral Edge-TPU this cell currently only runs on debian based systems

In [ ]:
# install dependencies
# ! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

# ! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

# ! sudo apt-get update

# ! sudo apt-get install edgetpu-compiler

! edgetpu_compiler {TFLITE_MODEL} -d --num_segments=1 -s -a

In [ ]:
test_img = os.path.join(paths.IMAGE_PATH, 'test_image.jpg')
command = f'python {pp.EDGE_TPU_DETECT} -m detect_edgetpu.tflite -l {pp.LABEL_FILE_LITE} -i {pp.TEST_IMAGE} -o {test_img} -t 0.2'
!{command}

# 11. Zip and Download Model in Colab

In [ ]:
!zip -r {os.path.join(GOOGLE_DRIVE_PATH, CUSTOM_MODEL_NAME + '.zip')} {CHECKPOINT_PATH}